In [46]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras_cv.models import StableDiffusion
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
import tensorflow.keras as keras

ModuleNotFoundError: No module named 'keras_cv.models.stable_diffusion.__internal__.utils'

In [23]:
for layer in diffusion_model.layers:
    print(layer.name)

input_2
dense
input_3
activation
padded_conv2d
dense_1
res_block
input_1
spatial_transformer
res_block_1
spatial_transformer_1
padded_conv2d_9
res_block_2
spatial_transformer_2
res_block_3
spatial_transformer_3
padded_conv2d_18
res_block_4
spatial_transformer_4
res_block_5
spatial_transformer_5
padded_conv2d_27
res_block_6
res_block_7
res_block_8
spatial_transformer_6
res_block_9
concatenate
res_block_10
concatenate_1
res_block_11
concatenate_2
res_block_12
upsample
concatenate_3
res_block_13
spatial_transformer_7
concatenate_4
res_block_14
spatial_transformer_8
concatenate_5
res_block_15
spatial_transformer_9
upsample_1
concatenate_6
res_block_16
spatial_transformer_10
concatenate_7
res_block_17
spatial_transformer_11
concatenate_8
res_block_18
spatial_transformer_12
upsample_2
concatenate_9
res_block_19
spatial_transformer_13
concatenate_10
res_block_20
spatial_transformer_14
concatenate_11
res_block_21
spatial_transformer_15
group_normalization_60
activation_67
padded_conv2d_83


In [56]:
Stable_diffusion = StableDiffusion(img_height=512, img_width=512)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [59]:
decoder = Stable_diffusion.decoder

In [29]:
diffusion_model = DiffusionModel(img_width = 512, img_height = 512, max_text_length=0)

In [53]:
diffusion_model.summary()

Model: "diffusion_model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 320)]        0           []                               
                                                                                                  
 dense_552 (Dense)              (None, 1280)         410880      ['input_11[0][0]']               
                                                                                                  
 input_12 (InputLayer)          [(None, 64, 64, 4)]  0           []                               
                                                                                                  
 activation_204 (Activation)    (None, 1280)         0           ['dense_552[0][0]']              
                                                                                  

In [55]:
diffusion_model.input_shape

[(None, 64, 64, 4), (None, 320), (None, 0, 768)]

In [63]:
# we are going to need to modify this code a lot 
def generate_pattern(layer_name, filter_index, size=64):
    # Build a model that outputs the activation
    # of the nth filter of the layer considered.
    layer_output = diffusion_model.get_layer(layer_name).output
    # Isolate the output 
    new_model = tf.keras.models.Model(inputs=diffusion_model.inputs, outputs=layer_output)
    
    # We start from a gray image with some uniform noise
    input_img_data = np.random.random((1, size, size, 4)) * 20 + 128.
    I = tf.Variable(input_img_data, name='image_var', dtype = 'float64')
    #I = preprocess_input(I_start) # only process once
    # Run gradient ascent for 40 steps
    eta = 1
    for i in range(100):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(I)
            # get variable to maximize 
            model_vals = new_model(I) 
            loss = tf.reduce_mean(model_vals[:, :, :, filter_index])

        # Compute the gradient of the input picture w.r.t. this loss
        # add this operation input to maximize
        grad_fn = tape.gradient(loss, I)
        # Normalization trick: we normalize the gradient
        grad_fn /= (tf.sqrt(tf.reduce_mean(tf.square(grad_fn))) + 1e-5) # mean L2 norm
        I -= grad_fn * eta # one iteration of maximizing
    # decode the resulting input image
    I = decoder(I)
    # return the numpy matrix so we can visualize 
    img = I.numpy()
    return img